In [1]:
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/gdrive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [8]:
cd gdrive/My\ Drive/음원순위_parameter tuning

/content/gdrive/My Drive/음원순위_parameter tuning


# 약간의 전처리 과정

In [0]:
df = pd.read_csv('NA_채우기_전.csv')
#df['you_rank_g'] = df['you_rank_g'].astype(str)
#df['previous_ranking'] = df['previous_ranking'].astype(str)

In [0]:
pd.set_option('display.max_columns', 999)

In [0]:
df.head()

,artist,name,rank_g,previous_ranking,genre,season,sex,song_type,active_type,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view,title_song,week,you_rank_g
0,(여자)아이들,LATATA,2,2.0,댄스,summer,female,K_POP,Group,203,24,20.0,9.2,NaN,-0.093812,2.3,NaN,NaN,NaN,NaN,True,2018.7.1,10
1,(여자)아이들,LATATA,2,2.0,댄스,summer,female,K_POP,Group,203,24,20.0,12.0,NaN,-0.093812,2.4,NaN,NaN,NaN,NaN,True,2018.6.24,10
2,(여자)아이들,LATATA,2,4.0,댄스,spring,female,K_POP,Group,203,24,59.0,34.9,NaN,0.012954,3.9,NaN,NaN,NaN,NaN,True,2018.5.13,10
3,(여자)아이들,LATATA,7,6.0,댄스,fall,female,K_POP,Group,203,24,58.0,11.1,4310000.0,-0.020121,4.6,NaN,NaN,NaN,NaN,False,2018.9.16,4
4,(여자)아이들,LATATA,4,4.0,댄스,summer,female,K_POP,Group,203,24,80.0,100.0,8730000.0,-0.093812,9.1,NaN,NaN,NaN,NaN,False,2018.8.12,2


In [0]:
# null값 제거
df.loc[df['total_view'].isnull(),'total_view'] = 0
df.loc[df['dc_total_numb'].isnull(),'dc_total_numb'] = 0
df.loc[df['dc_mean_views'].isnull(),'dc_mean_views'] = 0
df.loc[df['dc_mean_reccomend'].isnull(),'dc_mean_reccomend'] = 0
df.loc[df['drama_view'].isnull(),'drama_view'] = 0
# df.loc[df['previous_ranking'].isnull(),'previous_ranking'] = '10'

In [0]:
# string으로 바꿔주기
df['rank_g'] = df['rank_g'].apply(lambda x: str(int(x)))
df['you_rank_g'] = df['you_rank_g'].apply(lambda x: str(int(x)))

In [0]:
df = df.drop(columns = ['artist','name'])

In [0]:
# 장르 전부다 영어로
df.loc[df['genre'] == '댄스', 'genre'] = 'Dance'
df.loc[df['genre'] == '전체', 'genre'] = 'All'
df.loc[df['genre'] == '랩/힙합', 'genre'] = 'Rap/Hiphop'
df.loc[df['genre'] == '드라마', 'genre'] = 'Drama'
df.loc[df['genre'] == '인디', 'genre'] = 'Indi'
df.loc[df['genre'] == '팝', 'genre'] = 'Pop'
df.loc[df['genre'] == 'R&B/소울', 'genre'] = 'R&B/Soul'
df.loc[df['genre'] == '일렉트로니카', 'genre'] = 'Electronica'
df.loc[df['genre'] == '캐롤', 'genre'] = 'carol'
df.loc[df['genre'] == '락', 'genre'] = 'Rock'
df.loc[df['genre'] == '애니메이션/게임', 'genre'] = 'Ani/Game'
df.loc[df['genre'] == '발라드', 'genre'] = 'Ballad'
df.loc[df['genre'] == '해외영화', 'genre'] = 'Foreign Movie'
df.loc[df['genre'] == '트로트', 'genre'] = 'Trot'
df.loc[df['genre'] == '블루스/포크', 'genre'] = 'Blues/Folk'
df.loc[df['genre'] == '정통', 'genre'] = 'Jazz'

In [0]:
df = df.drop(columns = ['week','rank_g'])

In [0]:
df.to_csv('train_model_final_data1.csv')

In [0]:
df = pd.read_csv('final_data2.csv')

In [10]:
df.isnull().sum()

artist                  8
name                    0
rank_g                  0
you_rank_g              0
previous_ranking      846
genre                   0
season                  0
sex                     0
song_type               0
active_type             0
title_song              0
week                    0
runtime                 0
top_freq                0
gg_score                0
nv_score                0
total_view              0
season_genre_score      0
pd_score                0
dc_total_numb           0
dc_mean_reccomend       0
dc_mean_views           0
drama_view              0
dtype: int64

In [0]:
list_rank = []
for i in df['previous_ranking']:
    list_rank.append(str(i).replace('.0',''))

In [0]:
list_rank

['9',
 '7',
 '3',
 '8',
 '4',
 '5',
 '3',
 '9',
 '3',
 '1',
 '7',
 '7',
 '8',
 '9',
 '1',
 '4',
 '2',
 '4',
 '1',
 '8',
 '1',
 '4',
 '8',
 '1',
 '2',
 '1',
 '4',
 '1',
 '2',
 '3',
 '8',
 '5',
 '5',
 '3',
 '6',
 '6',
 '2',
 '6',
 '4',
 '6',
 '7',
 '2',
 '5',
 '5',
 '9',
 '3',
 '2',
 '2',
 '5',
 '3',
 '4',
 '6',
 '3',
 '3',
 '8',
 '8',
 '4',
 '1',
 '1',
 '1',
 '10',
 '2',
 '9',
 'nan',
 'nan',
 'nan',
 'nan',
 '5',
 '6',
 '7',
 '2',
 '8',
 '6',
 '2',
 '10',
 '5',
 '5',
 '7',
 '7',
 '9',
 '6',
 '9',
 '8',
 '3',
 '4',
 '8',
 '7',
 '6',
 '10',
 '7',
 '4',
 '10',
 '7',
 'nan',
 '10',
 'nan',
 '5',
 '10',
 '6',
 'nan',
 '10',
 '7',
 '3',
 '8',
 '4',
 '4',
 '4',
 '9',
 '3',
 '1',
 '7',
 '7',
 '6',
 '9',
 '1',
 '5',
 '2',
 '4',
 '1',
 '9',
 '1',
 '4',
 '1',
 '2',
 '1',
 '4',
 '1',
 '2',
 '3',
 '8',
 '5',
 '5',
 '3',
 '6',
 '6',
 '2',
 '7',
 '4',
 '6',
 '7',
 '2',
 '5',
 '5',
 '3',
 '2',
 '2',
 '5',
 '2',
 '4',
 '6',
 '3',
 '3',
 '8',
 '1',
 '3',
 '1',
 '8',
 '1',
 '5',
 '4',
 '10',
 '8',
 '5',


In [0]:
df['previous_ranking'] = list_rank

In [13]:
df

,artist,name,rank_g,you_rank_g,previous_ranking,genre,season,sex,song_type,active_type,title_song,week,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view
0,10cm,폰서트,10,6,9,Indi,spring,male,K_POP,Solo,1,2018.5.6,0.508876,0.240385,0.35,0.519574,0.800946,0.513550,0.862314,0.0,0.0,0.0,0.0
1,BLACKPINK,마지막처럼,7,2,7,Dance,spring,female,K_POP,Group,1,2018.5.6,0.553254,0.317308,0.09,0.492322,0.847722,0.551930,0.607430,0.0,0.0,0.0,0.0
2,Camila Cabello,Havana,3,6,3,Pop,spring,female,POP,Solo,1,2018.5.6,0.565089,0.855769,0.39,0.887520,0.808891,0.529333,0.000000,0.0,0.0,0.0,0.0
3,Carla Bruni,Stand By Your Man,8,8,8,Pop,spring,female,POP,Solo,1,2018.5.6,0.408284,0.038462,0.00,0.896350,0.786454,0.529333,0.659684,0.0,0.0,0.0,0.0
4,DEAN,instagram,4,8,4,R&B/Soul,spring,male,K_POP,Solo,1,2018.5.6,0.680473,0.250000,0.36,0.781824,0.788971,0.566807,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17158,창모,빌었어,8,10,8,Rap/Hiphop,spring,male,K_POP,Solo,0,2020.4.19,0.576923,0.192308,0.38,0.540225,0.000000,0.477572,0.480855,0.0,0.0,0.0,0.0
17159,폴킴,Hero,10,10,nan,R&B/Soul,spring,male,K_POP,Solo,0,2020.4.19,0.606509,0.000000,0.27,0.487808,0.000000,0.566807,0.677504,0.0,0.0,0.0,0.0
17160,폴킴,나의 봄의 이유,10,10,nan,R&B/Soul,spring,male,K_POP,Solo,0,2020.4.19,0.517751,0.000000,0.27,0.487808,0.000000,0.566807,0.677504,0.0,0.0,0.0,0.0
17161,폴킴,비,10,10,10,R&B/Soul,spring,male,K_POP,Solo,0,2020.4.19,0.627219,0.980769,0.27,0.487808,0.000000,0.566807,0.677504,0.0,0.0,0.0,0.0


In [0]:
# 2020.4.19를 테스트로 한다. 
test = df.loc[df['week'] == '2020.4.19']
train = df.loc[df['week'] != '2020.4.19']

In [0]:
# week 필요없음
train = train.drop(columns = ['week','artist','name'])
test = test.drop(columns = ['week','artist','name'])

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns = ['rank_g']),train['rank_g'], test_size = 0.3)

# 사용 예정 모델
- XGboost
- LightGBM
- RandomForest
- Catboost
- Logistic regression

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Logistic Regression

In [0]:
model = LogisticRegression(random_state=1)

In [0]:
stf = StratifiedKFold(n_splits = 2, shuffle = True, random_state = 1)

In [0]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')
score.mean()

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-pa

0.6997561050303264

In [0]:
model.fit(X_train,y_train)

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
pred_y = model.predict(X_test)

In [0]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.689


# RandomForest

In [0]:
model = RandomForestClassifier(random_state =1, n_estimators = 500)

In [0]:
stf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)

In [0]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

In [0]:
score.mean()

0.6888141663812283

In [0]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [0]:
pred_y = model.predict(X_test)

In [0]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.679


# Catboost

In [20]:
!pip install catboost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

     |████████████████████████████████| 64.8MB 47kB/s 


In [0]:
cat = [0,1,2,3,4,5,6]

In [22]:
X_train.head()

,you_rank_g,previous_ranking,genre,season,sex,song_type,active_type,title_song,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view
3374,10,7,Ballad,winter,male,K_POP,Solo,1,0.659763,0.384615,0.40,0.603454,0.000000,0.537976,0.673668,0.000000,0.000000,0.035303,0.0
14160,7,nan,Rap/Hiphop,winter,male,K_POP,Project,0,0.606509,0.057692,0.25,0.568711,0.798248,0.533239,0.473672,0.000000,0.000000,0.000000,0.0
7839,10,10,Ballad,summer,male,K_POP,Group,1,0.633136,0.336538,0.27,0.405580,0.000000,0.475960,0.170842,0.000000,0.000000,0.000000,0.0
11756,10,6,Blues/Folk,fall,mixed,K_POP,Duet,0,0.458580,0.076923,0.41,0.418520,0.000000,0.453998,0.361357,0.539749,0.172980,0.036241,0.0
13739,10,nan,Ballad,winter,male,K_POP,Solo,1,0.612426,0.000000,0.28,0.310953,0.000000,0.537976,0.215217,0.330922,0.067036,0.020418,0.0


In [0]:
model = CatBoostClassifier(n_estimators= 5000, learning_rate = 0.001, border_count = 7, depth = 10, random_state= 1,cat_features = cat)

In [0]:
regression = CatBoostRegressor(task_type= 'GPU',n_estimators= 5000, learning_rate = 0.001, border_count = 7, depth = 10, random_state= 1,cat_features = cat)

In [24]:
regression.fit(X_train,y_train)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1:	learn: 3.1245204	total: 274ms	remaining: 11m 24s
2:	learn: 3.1221477	total: 456ms	remaining: 12m 39s
3:	learn: 3.1198101	total: 560ms	remaining: 11m 38s
4:	learn: 3.1175053	total: 605ms	remaining: 10m 4s
5:	learn: 3.1151464	total: 750ms	remaining: 10m 24s
6:	learn: 3.1127992	total: 899ms	remaining: 10m 40s
7:	learn: 3.1104386	total: 995ms	remaining: 10m 20s
8:	learn: 3.1081216	total: 1.05s	remaining: 9m 40s
9:	learn: 3.1057678	total: 1.2s	remaining: 10m
10:	learn: 3.1034732	total: 1.27s	remaining: 9m 34s
11:	learn: 3.1011221	total: 1.35s	remaining: 9m 22s
12:	learn: 3.0988293	total: 1.4s	remaining: 8m 55s
13:	learn: 3.0965558	total: 1.46s	remaining: 8m 41s
14:	learn: 3.0942704	total: 1.55s	remaining: 8m 35s
15:	learn: 3.0919447	total: 1.61s	remaining: 8m 21s
16:	learn: 3.0896397	total: 1.7s	remaining: 8m 18s
17:	learn: 3.0873428	total: 1.75s	remaining: 8m 3s
18:	learn: 3.0850393	total: 1.83s	remaining: 7m 59s
19:	learn: 3.0827389	total: 1.92s	rema

In [0]:
model.fit(X_train,y_train)

0:	learn: 2.2977546	total: 379ms	remaining: 31m 34s
1:	learn: 2.2930750	total: 1.25s	remaining: 52m 18s
2:	learn: 2.2883567	total: 1.84s	remaining: 51m 9s
3:	learn: 2.2834654	total: 2.51s	remaining: 52m 17s
4:	learn: 2.2789367	total: 2.64s	remaining: 44m 1s
5:	learn: 2.2744065	total: 2.96s	remaining: 41m
6:	learn: 2.2697387	total: 3.2s	remaining: 38m 4s
7:	learn: 2.2652545	total: 3.32s	remaining: 34m 32s
8:	learn: 2.2605929	total: 3.63s	remaining: 33m 33s
9:	learn: 2.2561825	total: 4.1s	remaining: 34m 6s
10:	learn: 2.2518465	total: 4.39s	remaining: 33m 13s
11:	learn: 2.2474560	total: 4.71s	remaining: 32m 39s
12:	learn: 2.2430599	total: 5.1s	remaining: 32m 37s
13:	learn: 2.2387091	total: 5.41s	remaining: 32m 5s
14:	learn: 2.2343904	total: 5.75s	remaining: 31m 52s
15:	learn: 2.2300750	total: 6.2s	remaining: 32m 9s
16:	learn: 2.2259067	total: 6.67s	remaining: 32m 34s
17:	learn: 2.2217668	total: 7.05s	remaining: 32m 31s
18:	learn: 2.2176539	total: 7.55s	remaining: 32m 59s
19:	learn: 2.2137

156:	learn: 1.8209990	total: 47.9s	remaining: 24m 37s
157:	learn: 1.8188109	total: 48.2s	remaining: 24m 38s
158:	learn: 1.8168825	total: 48.3s	remaining: 24m 30s
159:	learn: 1.8150032	total: 48.4s	remaining: 24m 23s
160:	learn: 1.8129070	total: 48.7s	remaining: 24m 23s
161:	learn: 1.8110979	total: 48.8s	remaining: 24m 16s
162:	learn: 1.8090569	total: 49.1s	remaining: 24m 15s
163:	learn: 1.8070351	total: 49.4s	remaining: 24m 15s
164:	learn: 1.8050870	total: 49.6s	remaining: 24m 13s
165:	learn: 1.8030968	total: 49.8s	remaining: 24m 10s
166:	learn: 1.8010542	total: 50.2s	remaining: 24m 11s
167:	learn: 1.7990204	total: 50.4s	remaining: 24m 10s
168:	learn: 1.7970429	total: 50.9s	remaining: 24m 15s
169:	learn: 1.7950915	total: 51s	remaining: 24m 10s
170:	learn: 1.7930587	total: 51.5s	remaining: 24m 14s
171:	learn: 1.7911389	total: 51.7s	remaining: 24m 10s
172:	learn: 1.7891269	total: 52.1s	remaining: 24m 14s
173:	learn: 1.7873463	total: 52.2s	remaining: 24m 8s
174:	learn: 1.7855510	total: 52

308:	learn: 1.5715688	total: 1m 30s	remaining: 22m 50s
309:	learn: 1.5701893	total: 1m 30s	remaining: 22m 51s
310:	learn: 1.5689675	total: 1m 30s	remaining: 22m 48s
311:	learn: 1.5676427	total: 1m 31s	remaining: 22m 47s
312:	learn: 1.5664206	total: 1m 31s	remaining: 22m 44s
313:	learn: 1.5651418	total: 1m 31s	remaining: 22m 45s
314:	learn: 1.5638959	total: 1m 31s	remaining: 22m 43s
315:	learn: 1.5626713	total: 1m 31s	remaining: 22m 40s
316:	learn: 1.5613576	total: 1m 32s	remaining: 22m 40s
317:	learn: 1.5599716	total: 1m 32s	remaining: 22m 40s
318:	learn: 1.5586239	total: 1m 32s	remaining: 22m 40s
319:	learn: 1.5573577	total: 1m 33s	remaining: 22m 41s
320:	learn: 1.5561199	total: 1m 33s	remaining: 22m 40s
321:	learn: 1.5547907	total: 1m 33s	remaining: 22m 40s
322:	learn: 1.5535061	total: 1m 33s	remaining: 22m 40s
323:	learn: 1.5521331	total: 1m 34s	remaining: 22m 41s
324:	learn: 1.5509067	total: 1m 34s	remaining: 22m 39s
325:	learn: 1.5496060	total: 1m 34s	remaining: 22m 41s
326:	learn

458:	learn: 1.4057682	total: 2m 16s	remaining: 22m 27s
459:	learn: 1.4047761	total: 2m 16s	remaining: 22m 27s
460:	learn: 1.4038710	total: 2m 16s	remaining: 22m 25s
461:	learn: 1.4030188	total: 2m 16s	remaining: 22m 24s
462:	learn: 1.4020871	total: 2m 17s	remaining: 22m 23s
463:	learn: 1.4011658	total: 2m 17s	remaining: 22m 22s
464:	learn: 1.4002722	total: 2m 17s	remaining: 22m 23s
465:	learn: 1.3993234	total: 2m 18s	remaining: 22m 24s
466:	learn: 1.3984208	total: 2m 18s	remaining: 22m 23s
467:	learn: 1.3975242	total: 2m 18s	remaining: 22m 22s
468:	learn: 1.3965834	total: 2m 19s	remaining: 22m 22s
469:	learn: 1.3956930	total: 2m 19s	remaining: 22m 22s
470:	learn: 1.3947368	total: 2m 19s	remaining: 22m 23s
471:	learn: 1.3938283	total: 2m 20s	remaining: 22m 23s
472:	learn: 1.3928989	total: 2m 20s	remaining: 22m 23s
473:	learn: 1.3921256	total: 2m 20s	remaining: 22m 21s
474:	learn: 1.3912638	total: 2m 20s	remaining: 22m 21s
475:	learn: 1.3902788	total: 2m 21s	remaining: 22m 21s
476:	learn

609:	learn: 1.2853307	total: 3m 1s	remaining: 21m 44s
610:	learn: 1.2846591	total: 3m 1s	remaining: 21m 44s
611:	learn: 1.2839489	total: 3m 1s	remaining: 21m 43s
612:	learn: 1.2833781	total: 3m 1s	remaining: 21m 41s
613:	learn: 1.2827532	total: 3m 1s	remaining: 21m 39s
614:	learn: 1.2820957	total: 3m 2s	remaining: 21m 38s
615:	learn: 1.2814553	total: 3m 2s	remaining: 21m 36s
616:	learn: 1.2808136	total: 3m 2s	remaining: 21m 36s
617:	learn: 1.2801361	total: 3m 2s	remaining: 21m 36s
618:	learn: 1.2794227	total: 3m 3s	remaining: 21m 36s
619:	learn: 1.2788289	total: 3m 3s	remaining: 21m 34s
620:	learn: 1.2781060	total: 3m 3s	remaining: 21m 35s
621:	learn: 1.2774029	total: 3m 4s	remaining: 21m 36s
622:	learn: 1.2767073	total: 3m 4s	remaining: 21m 37s
623:	learn: 1.2759880	total: 3m 5s	remaining: 21m 37s
624:	learn: 1.2753102	total: 3m 5s	remaining: 21m 38s
625:	learn: 1.2746451	total: 3m 5s	remaining: 21m 38s
626:	learn: 1.2739279	total: 3m 6s	remaining: 21m 38s
627:	learn: 1.2733001	total:

...

4274:	learn: 0.7285735	total: 42m 12s	remaining: 7m 9s
4275:	learn: 0.7285506	total: 42m 13s	remaining: 7m 8s
4276:	learn: 0.7285257	total: 42m 14s	remaining: 7m 8s
4277:	learn: 0.7284961	total: 42m 15s	remaining: 7m 7s
4278:	learn: 0.7284547	total: 42m 16s	remaining: 7m 7s
4279:	learn: 0.7284346	total: 42m 16s	remaining: 7m 6s
4280:	learn: 0.7283907	total: 42m 17s	remaining: 7m 6s
4281:	learn: 0.7283550	total: 42m 19s	remaining: 7m 5s
4282:	learn: 0.7283013	total: 42m 20s	remaining: 7m 5s
4283:	learn: 0.7282742	total: 42m 22s	remaining: 7m 4s
4284:	learn: 0.7282490	total: 42m 23s	remaining: 7m 4s
4285:	learn: 0.7282082	total: 42m 24s	remaining: 7m 3s
4286:	learn: 0.7281767	total: 42m 25s	remaining: 7m 3s
4287:	learn: 0.7281365	total: 42m 26s	remaining: 7m 2s
4288:	learn: 0.7281003	total: 42m 27s	remaining: 7m 2s
4289:	learn: 0.7280698	total: 42m 28s	remaining: 7m 1s
4290:	learn: 0.7280297	total: 42m 29s	remaining: 7m 1s
4291:	learn: 0.7280027	total: 42m 30s	remaining: 7m
4292:	learn: 

4422:	learn: 0.7238149	total: 44m 20s	remaining: 5m 47s
4423:	learn: 0.7238102	total: 44m 20s	remaining: 5m 46s
4424:	learn: 0.7237747	total: 44m 21s	remaining: 5m 45s
4425:	learn: 0.7237568	total: 44m 22s	remaining: 5m 45s
4426:	learn: 0.7237135	total: 44m 22s	remaining: 5m 44s
4427:	learn: 0.7236794	total: 44m 23s	remaining: 5m 44s
4428:	learn: 0.7236574	total: 44m 24s	remaining: 5m 43s
4429:	learn: 0.7236221	total: 44m 24s	remaining: 5m 42s
4430:	learn: 0.7235810	total: 44m 25s	remaining: 5m 42s
4431:	learn: 0.7235442	total: 44m 26s	remaining: 5m 41s
4432:	learn: 0.7235067	total: 44m 27s	remaining: 5m 41s
4433:	learn: 0.7234694	total: 44m 27s	remaining: 5m 40s
4434:	learn: 0.7234432	total: 44m 28s	remaining: 5m 39s
4435:	learn: 0.7233973	total: 44m 29s	remaining: 5m 39s
4436:	learn: 0.7233537	total: 44m 30s	remaining: 5m 38s
4437:	learn: 0.7233311	total: 44m 30s	remaining: 5m 38s
4438:	learn: 0.7233084	total: 44m 31s	remaining: 5m 37s
4439:	learn: 0.7232803	total: 44m 32s	remaining:

4571:	learn: 0.7192535	total: 46m 19s	remaining: 4m 20s
4572:	learn: 0.7192142	total: 46m 20s	remaining: 4m 19s
4573:	learn: 0.7191892	total: 46m 20s	remaining: 4m 18s
4574:	learn: 0.7191833	total: 46m 20s	remaining: 4m 18s
4575:	learn: 0.7191383	total: 46m 21s	remaining: 4m 17s
4576:	learn: 0.7191169	total: 46m 22s	remaining: 4m 17s
4577:	learn: 0.7190746	total: 46m 22s	remaining: 4m 16s
4578:	learn: 0.7190529	total: 46m 23s	remaining: 4m 15s
4579:	learn: 0.7190117	total: 46m 24s	remaining: 4m 15s
4580:	learn: 0.7189872	total: 46m 25s	remaining: 4m 14s
4581:	learn: 0.7189654	total: 46m 25s	remaining: 4m 14s
4582:	learn: 0.7189435	total: 46m 26s	remaining: 4m 13s
4583:	learn: 0.7189087	total: 46m 27s	remaining: 4m 12s
4584:	learn: 0.7188846	total: 46m 27s	remaining: 4m 12s
4585:	learn: 0.7188667	total: 46m 28s	remaining: 4m 11s
4586:	learn: 0.7188280	total: 46m 29s	remaining: 4m 11s
4587:	learn: 0.7188037	total: 46m 30s	remaining: 4m 10s
4588:	learn: 0.7187673	total: 46m 30s	remaining:

4719:	learn: 0.7148918	total: 48m 6s	remaining: 2m 51s
4720:	learn: 0.7148694	total: 48m 7s	remaining: 2m 50s
4721:	learn: 0.7148455	total: 48m 8s	remaining: 2m 50s
4722:	learn: 0.7148012	total: 48m 8s	remaining: 2m 49s
4723:	learn: 0.7147660	total: 48m 9s	remaining: 2m 48s
4724:	learn: 0.7147389	total: 48m 10s	remaining: 2m 48s
4725:	learn: 0.7147027	total: 48m 10s	remaining: 2m 47s
4726:	learn: 0.7146841	total: 48m 11s	remaining: 2m 47s
4727:	learn: 0.7146461	total: 48m 12s	remaining: 2m 46s
4728:	learn: 0.7146224	total: 48m 13s	remaining: 2m 45s
4729:	learn: 0.7145669	total: 48m 13s	remaining: 2m 45s
4730:	learn: 0.7145367	total: 48m 14s	remaining: 2m 44s
4731:	learn: 0.7145168	total: 48m 15s	remaining: 2m 43s
4732:	learn: 0.7144778	total: 48m 16s	remaining: 2m 43s
4733:	learn: 0.7144431	total: 48m 16s	remaining: 2m 42s
4734:	learn: 0.7144100	total: 48m 17s	remaining: 2m 42s
4735:	learn: 0.7143740	total: 48m 18s	remaining: 2m 41s
4736:	learn: 0.7143301	total: 48m 19s	remaining: 2m 4

4867:	learn: 0.7105757	total: 49m 54s	remaining: 1m 21s
4868:	learn: 0.7105540	total: 49m 55s	remaining: 1m 20s
4869:	learn: 0.7105389	total: 49m 56s	remaining: 1m 19s
4870:	learn: 0.7105225	total: 49m 56s	remaining: 1m 19s
4871:	learn: 0.7104988	total: 49m 57s	remaining: 1m 18s
4872:	learn: 0.7104630	total: 49m 58s	remaining: 1m 18s
4873:	learn: 0.7104292	total: 49m 59s	remaining: 1m 17s
4874:	learn: 0.7103904	total: 49m 59s	remaining: 1m 16s
4875:	learn: 0.7103664	total: 50m	remaining: 1m 16s
4876:	learn: 0.7103290	total: 50m 1s	remaining: 1m 15s
4877:	learn: 0.7102948	total: 50m 2s	remaining: 1m 15s
4878:	learn: 0.7102679	total: 50m 2s	remaining: 1m 14s
4879:	learn: 0.7102404	total: 50m 3s	remaining: 1m 13s
4880:	learn: 0.7102045	total: 50m 4s	remaining: 1m 13s
4881:	learn: 0.7101834	total: 50m 5s	remaining: 1m 12s
4882:	learn: 0.7101492	total: 50m 6s	remaining: 1m 12s
4883:	learn: 0.7101312	total: 50m 6s	remaining: 1m 11s
4884:	learn: 0.7100987	total: 50m 7s	remaining: 1m 10s
4885:

In [25]:
pred_y = model.predict(X_test)

NameError: ignored

In [0]:
pred_y = regression.predict(X_test)

In [27]:
pred_y

array([5.76584006, 3.70088165, 3.5060992 , ..., 8.56427986, 2.98082696,
       5.2984483 ])

In [0]:
# 최종 결과
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.726


In [0]:
import joblib

In [0]:
pred_y = model.predict(X_test)

In [0]:
df['rank_g_pred'] = model.predict(df.drop(columns = ['rank_g','artist','name','week']))

In [0]:
df['rank_g_pred'] = regression.predict(df.drop(columns = ['rank_g','artist','name','week']))

In [0]:
a = df.drop(columns = ['artist','week','rank_g_pred','rank_g','name'])

In [29]:
df

,artist,name,rank_g,you_rank_g,previous_ranking,genre,season,sex,song_type,active_type,title_song,week,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view,rank_g_pred
0,10cm,폰서트,10,6,9,Indi,spring,male,K_POP,Solo,1,2018.5.6,0.508876,0.240385,0.35,0.519574,0.800946,0.513550,0.862314,0.0,0.0,0.0,0.0,8.641867
1,BLACKPINK,마지막처럼,7,2,7,Dance,spring,female,K_POP,Group,1,2018.5.6,0.553254,0.317308,0.09,0.492322,0.847722,0.551930,0.607430,0.0,0.0,0.0,0.0,7.177732
2,Camila Cabello,Havana,3,6,3,Pop,spring,female,POP,Solo,1,2018.5.6,0.565089,0.855769,0.39,0.887520,0.808891,0.529333,0.000000,0.0,0.0,0.0,0.0,4.163253
3,Carla Bruni,Stand By Your Man,8,8,8,Pop,spring,female,POP,Solo,1,2018.5.6,0.408284,0.038462,0.00,0.896350,0.786454,0.529333,0.659684,0.0,0.0,0.0,0.0,8.227471
4,DEAN,instagram,4,8,4,R&B/Soul,spring,male,K_POP,Solo,1,2018.5.6,0.680473,0.250000,0.36,0.781824,0.788971,0.566807,0.000000,0.0,0.0,0.0,0.0,4.464440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17158,창모,빌었어,8,10,8,Rap/Hiphop,spring,male,K_POP,Solo,0,2020.4.19,0.576923,0.192308,0.38,0.540225,0.000000,0.477572,0.480855,0.0,0.0,0.0,0.0,8.253079
17159,폴킴,Hero,10,10,nan,R&B/Soul,spring,male,K_POP,Solo,0,2020.4.19,0.606509,0.000000,0.27,0.487808,0.000000,0.566807,0.677504,0.0,0.0,0.0,0.0,8.646731
17160,폴킴,나의 봄의 이유,10,10,nan,R&B/Soul,spring,male,K_POP,Solo,0,2020.4.19,0.517751,0.000000,0.27,0.487808,0.000000,0.566807,0.677504,0.0,0.0,0.0,0.0,8.646731
17161,폴킴,비,10,10,10,R&B/Soul,spring,male,K_POP,Solo,0,2020.4.19,0.627219,0.980769,0.27,0.487808,0.000000,0.566807,0.677504,0.0,0.0,0.0,0.0,9.918421


In [0]:
# 최종 결과 저장
df.to_csv('최종 결과_그래프_final.csv')

In [0]:
a.to_csv('최종결과_model_final.csv')

In [0]:
# 
print("F1 : %.3f" % f1_score(df['rank_g_pred'], df['rank_g'], average = 'micro'))

F1 : 0.736


In [0]:
import joblib

In [0]:
joblib.dump(model,'cpu_version_final.pkl')

['cpu_version_final.pkl']

In [0]:
df = pd.read_csv('log_scaled_data_final.csv')

In [0]:
df['title_song'] = df['title_song'].apply(lambda x: 1 if x == True else 0)

In [0]:
df

,rank_g,you_rank_g,previous_ranking,genre,season,sex,song_type,active_type,title_song,week,artist,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view,rank_g_pred
0,2,10,2,Dance,summer,female,K_POP,Group,1,2018.7.1,(여자)아이들,0.523669,0.221154,0.20,0.503213,0.000000,0.394525,0.258698,0.0,0.0,0.0,0.0,2
1,2,10,2,Dance,summer,female,K_POP,Group,1,2018.6.24,(여자)아이들,0.523669,0.221154,0.20,0.555771,0.000000,0.394525,0.265167,0.0,0.0,0.0,0.0,2
2,2,10,4,Dance,spring,female,K_POP,Group,1,2018.5.13,(여자)아이들,0.523669,0.221154,0.59,0.775871,0.000000,0.551930,0.344354,0.0,0.0,0.0,0.0,4
3,7,4,6,Dance,fall,female,K_POP,Group,0,2018.9.16,(여자)아이들,0.523669,0.221154,0.58,0.540225,0.831011,0.503168,0.373287,0.0,0.0,0.0,0.0,7
4,4,2,4,Dance,summer,female,K_POP,Group,0,2018.8.12,(여자)아이들,0.523669,0.221154,0.80,1.000000,0.869406,0.394525,0.501078,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17358,9,10,1,Indi,summer,female,K_POP,Solo,1,2019.6.2,히키,0.674556,0.067308,0.41,1.000000,0.000000,0.501742,0.000000,0.0,0.0,0.0,0.0,1
17359,10,10,1,Indi,summer,female,K_POP,Solo,1,2019.7.21,히키,0.674556,0.067308,0.22,0.620179,0.000000,0.501742,0.000000,0.0,0.0,0.0,0.0,1
17360,10,10,1,Indi,summer,female,K_POP,Solo,1,2019.6.9,히키,0.674556,0.067308,0.41,0.834246,0.000000,0.501742,0.000000,0.0,0.0,0.0,0.0,1
17361,10,10,1,Indi,summer,female,K_POP,Solo,1,2019.6.30,히키,0.674556,0.067308,0.32,0.762168,0.000000,0.501742,0.000000,0.0,0.0,0.0,0.0,1


In [0]:
# 2020.4.19를 테스트로 한다. 
test = df.loc[df['week'] == '2020.4.19']
train = df.loc[df['week'] != '2020.4.19']

In [0]:
model.fit(X_train,y_train)

0:	learn: 2.2979395	total: 549ms	remaining: 45m 45s
1:	learn: 2.2934006	total: 1.01s	remaining: 41m 59s
2:	learn: 2.2890681	total: 1.49s	remaining: 41m 26s
3:	learn: 2.2847199	total: 1.85s	remaining: 38m 30s
4:	learn: 2.2803560	total: 2.01s	remaining: 33m 25s
5:	learn: 2.2759706	total: 2.32s	remaining: 32m 12s
6:	learn: 2.2716765	total: 2.64s	remaining: 31m 22s
7:	learn: 2.2672927	total: 2.92s	remaining: 30m 22s
8:	learn: 2.2629703	total: 3.19s	remaining: 29m 28s
9:	learn: 2.2587637	total: 3.64s	remaining: 30m 14s
10:	learn: 2.2543262	total: 4.01s	remaining: 30m 21s
11:	learn: 2.2502700	total: 4.34s	remaining: 30m 6s
12:	learn: 2.2461887	total: 4.65s	remaining: 29m 45s
13:	learn: 2.2423199	total: 5.01s	remaining: 29m 45s
14:	learn: 2.2383260	total: 5.24s	remaining: 29m 1s
15:	learn: 2.2342064	total: 5.53s	remaining: 28m 41s
16:	learn: 2.2301702	total: 5.82s	remaining: 28m 24s
17:	learn: 2.2261675	total: 6.16s	remaining: 28m 25s
18:	learn: 2.2225413	total: 6.22s	remaining: 27m 11s
19:	l

156:	learn: 1.8418999	total: 41.7s	remaining: 21m 25s
157:	learn: 1.8399205	total: 41.8s	remaining: 21m 20s
158:	learn: 1.8379095	total: 42.2s	remaining: 21m 24s
159:	learn: 1.8358589	total: 42.5s	remaining: 21m 26s
160:	learn: 1.8338673	total: 42.9s	remaining: 21m 28s
161:	learn: 1.8318598	total: 43.2s	remaining: 21m 28s
162:	learn: 1.8298504	total: 43.5s	remaining: 21m 29s
163:	learn: 1.8279556	total: 43.5s	remaining: 21m 23s
164:	learn: 1.8261245	total: 43.9s	remaining: 21m 26s
165:	learn: 1.8242203	total: 44.2s	remaining: 21m 27s
166:	learn: 1.8222463	total: 44.6s	remaining: 21m 30s
167:	learn: 1.8203657	total: 44.8s	remaining: 21m 27s
168:	learn: 1.8183081	total: 45s	remaining: 21m 26s
169:	learn: 1.8163835	total: 45.1s	remaining: 21m 21s
170:	learn: 1.8144422	total: 45.5s	remaining: 21m 24s
171:	learn: 1.8124753	total: 45.8s	remaining: 21m 24s
172:	learn: 1.8105875	total: 46.1s	remaining: 21m 26s
173:	learn: 1.8086502	total: 46.4s	remaining: 21m 27s
174:	learn: 1.8066638	total: 4

309:	learn: 1.5994259	total: 1m 20s	remaining: 20m 20s
310:	learn: 1.5981697	total: 1m 21s	remaining: 20m 22s
311:	learn: 1.5969152	total: 1m 21s	remaining: 20m 23s
312:	learn: 1.5957390	total: 1m 21s	remaining: 20m 20s
313:	learn: 1.5945072	total: 1m 21s	remaining: 20m 19s
314:	learn: 1.5931845	total: 1m 22s	remaining: 20m 20s
315:	learn: 1.5919567	total: 1m 22s	remaining: 20m 20s
316:	learn: 1.5906145	total: 1m 22s	remaining: 20m 21s
317:	learn: 1.5893982	total: 1m 22s	remaining: 20m 19s
318:	learn: 1.5882112	total: 1m 23s	remaining: 20m 20s
319:	learn: 1.5869607	total: 1m 23s	remaining: 20m 21s
320:	learn: 1.5857192	total: 1m 23s	remaining: 20m 21s
321:	learn: 1.5846081	total: 1m 23s	remaining: 20m 18s
322:	learn: 1.5833278	total: 1m 24s	remaining: 20m 18s
323:	learn: 1.5820917	total: 1m 24s	remaining: 20m 19s
324:	learn: 1.5808879	total: 1m 24s	remaining: 20m 20s
325:	learn: 1.5796664	total: 1m 25s	remaining: 20m 20s
326:	learn: 1.5784662	total: 1m 25s	remaining: 20m 21s
327:	learn

459:	learn: 1.4405674	total: 2m 6s	remaining: 20m 50s
460:	learn: 1.4396968	total: 2m 7s	remaining: 20m 51s
461:	learn: 1.4388446	total: 2m 7s	remaining: 20m 49s
462:	learn: 1.4379191	total: 2m 7s	remaining: 20m 49s
463:	learn: 1.4369822	total: 2m 7s	remaining: 20m 50s
464:	learn: 1.4360774	total: 2m 8s	remaining: 20m 50s
465:	learn: 1.4351530	total: 2m 8s	remaining: 20m 51s
466:	learn: 1.4342506	total: 2m 8s	remaining: 20m 51s
467:	learn: 1.4333178	total: 2m 9s	remaining: 20m 51s
468:	learn: 1.4323812	total: 2m 9s	remaining: 20m 53s
469:	learn: 1.4314990	total: 2m 10s	remaining: 20m 53s
470:	learn: 1.4305813	total: 2m 10s	remaining: 20m 53s
471:	learn: 1.4296228	total: 2m 10s	remaining: 20m 54s
472:	learn: 1.4287732	total: 2m 11s	remaining: 20m 54s
473:	learn: 1.4279124	total: 2m 11s	remaining: 20m 55s
474:	learn: 1.4270971	total: 2m 11s	remaining: 20m 52s
475:	learn: 1.4262539	total: 2m 12s	remaining: 20m 54s
476:	learn: 1.4254380	total: 2m 12s	remaining: 20m 53s
477:	learn: 1.424567

610:	learn: 1.3246519	total: 2m 57s	remaining: 21m 16s
611:	learn: 1.3239682	total: 2m 58s	remaining: 21m 17s
612:	learn: 1.3233223	total: 2m 58s	remaining: 21m 17s
613:	learn: 1.3227099	total: 2m 58s	remaining: 21m 15s
614:	learn: 1.3221331	total: 2m 58s	remaining: 21m 13s
615:	learn: 1.3214893	total: 2m 59s	remaining: 21m 14s
616:	learn: 1.3208100	total: 2m 59s	remaining: 21m 14s
617:	learn: 1.3201453	total: 2m 59s	remaining: 21m 14s
618:	learn: 1.3194707	total: 3m	remaining: 21m 15s
619:	learn: 1.3187943	total: 3m	remaining: 21m 16s
620:	learn: 1.3180887	total: 3m 1s	remaining: 21m 17s
621:	learn: 1.3174332	total: 3m 1s	remaining: 21m 18s
622:	learn: 1.3167804	total: 3m 2s	remaining: 21m 19s
623:	learn: 1.3161520	total: 3m 2s	remaining: 21m 20s
624:	learn: 1.3155089	total: 3m 2s	remaining: 21m 20s
625:	learn: 1.3148781	total: 3m 3s	remaining: 21m 18s
626:	learn: 1.3142208	total: 3m 3s	remaining: 21m 17s
627:	learn: 1.3135897	total: 3m 3s	remaining: 21m 18s
628:	learn: 1.3129421	tota

...

4275:	learn: 0.7821993	total: 40m 24s	remaining: 6m 50s
4276:	learn: 0.7821596	total: 40m 25s	remaining: 6m 50s
4277:	learn: 0.7821292	total: 40m 26s	remaining: 6m 49s
4278:	learn: 0.7820958	total: 40m 27s	remaining: 6m 49s
4279:	learn: 0.7820373	total: 40m 28s	remaining: 6m 48s
4280:	learn: 0.7820105	total: 40m 29s	remaining: 6m 48s
4281:	learn: 0.7819807	total: 40m 29s	remaining: 6m 47s
4282:	learn: 0.7819489	total: 40m 30s	remaining: 6m 47s
4283:	learn: 0.7819212	total: 40m 31s	remaining: 6m 46s
4284:	learn: 0.7818674	total: 40m 32s	remaining: 6m 45s
4285:	learn: 0.7818403	total: 40m 33s	remaining: 6m 45s
4286:	learn: 0.7817851	total: 40m 33s	remaining: 6m 44s
4287:	learn: 0.7817472	total: 40m 34s	remaining: 6m 44s
4288:	learn: 0.7817215	total: 40m 35s	remaining: 6m 43s
4289:	learn: 0.7816719	total: 40m 36s	remaining: 6m 43s
4290:	learn: 0.7816157	total: 40m 37s	remaining: 6m 42s
4291:	learn: 0.7815785	total: 40m 37s	remaining: 6m 42s
4292:	learn: 0.7815516	total: 40m 38s	remaining:

4423:	learn: 0.7769188	total: 42m 25s	remaining: 5m 31s
4424:	learn: 0.7769005	total: 42m 26s	remaining: 5m 30s
4425:	learn: 0.7768572	total: 42m 27s	remaining: 5m 30s
4426:	learn: 0.7768080	total: 42m 28s	remaining: 5m 29s
4427:	learn: 0.7767833	total: 42m 28s	remaining: 5m 29s
4428:	learn: 0.7767391	total: 42m 29s	remaining: 5m 28s
4429:	learn: 0.7766894	total: 42m 30s	remaining: 5m 28s
4430:	learn: 0.7766391	total: 42m 31s	remaining: 5m 27s
4431:	learn: 0.7765836	total: 42m 32s	remaining: 5m 27s
4432:	learn: 0.7765582	total: 42m 33s	remaining: 5m 26s
4433:	learn: 0.7765420	total: 42m 34s	remaining: 5m 26s
4434:	learn: 0.7765018	total: 42m 34s	remaining: 5m 25s
4435:	learn: 0.7764832	total: 42m 35s	remaining: 5m 24s
4436:	learn: 0.7764580	total: 42m 36s	remaining: 5m 24s
4437:	learn: 0.7764161	total: 42m 37s	remaining: 5m 23s
4438:	learn: 0.7763664	total: 42m 38s	remaining: 5m 23s
4439:	learn: 0.7763433	total: 42m 39s	remaining: 5m 22s
4440:	learn: 0.7763031	total: 42m 40s	remaining:

4571:	learn: 0.7718917	total: 44m 21s	remaining: 4m 9s
4572:	learn: 0.7718589	total: 44m 22s	remaining: 4m 8s
4573:	learn: 0.7718307	total: 44m 23s	remaining: 4m 8s
4574:	learn: 0.7717826	total: 44m 23s	remaining: 4m 7s
4575:	learn: 0.7717524	total: 44m 24s	remaining: 4m 6s
4576:	learn: 0.7717235	total: 44m 25s	remaining: 4m 6s
4577:	learn: 0.7716896	total: 44m 26s	remaining: 4m 5s
4578:	learn: 0.7716615	total: 44m 27s	remaining: 4m 5s
4579:	learn: 0.7716242	total: 44m 28s	remaining: 4m 4s
4580:	learn: 0.7715824	total: 44m 28s	remaining: 4m 4s
4581:	learn: 0.7715563	total: 44m 29s	remaining: 4m 3s
4582:	learn: 0.7715202	total: 44m 30s	remaining: 4m 3s
4583:	learn: 0.7714931	total: 44m 31s	remaining: 4m 2s
4584:	learn: 0.7714603	total: 44m 32s	remaining: 4m 1s
4585:	learn: 0.7714244	total: 44m 33s	remaining: 4m 1s
4586:	learn: 0.7713934	total: 44m 33s	remaining: 4m
4587:	learn: 0.7713419	total: 44m 34s	remaining: 4m
4588:	learn: 0.7712934	total: 44m 35s	remaining: 3m 59s
4589:	learn: 0.

4719:	learn: 0.7667026	total: 46m 18s	remaining: 2m 44s
4720:	learn: 0.7666495	total: 46m 19s	remaining: 2m 44s
4721:	learn: 0.7666045	total: 46m 20s	remaining: 2m 43s
4722:	learn: 0.7666044	total: 46m 20s	remaining: 2m 43s
4723:	learn: 0.7665745	total: 46m 21s	remaining: 2m 42s
4724:	learn: 0.7665504	total: 46m 22s	remaining: 2m 41s
4725:	learn: 0.7665004	total: 46m 22s	remaining: 2m 41s
4726:	learn: 0.7664824	total: 46m 23s	remaining: 2m 40s
4727:	learn: 0.7664578	total: 46m 24s	remaining: 2m 40s
4728:	learn: 0.7664304	total: 46m 25s	remaining: 2m 39s
4729:	learn: 0.7664014	total: 46m 25s	remaining: 2m 39s
4730:	learn: 0.7663637	total: 46m 26s	remaining: 2m 38s
4731:	learn: 0.7663380	total: 46m 27s	remaining: 2m 37s
4732:	learn: 0.7663071	total: 46m 28s	remaining: 2m 37s
4733:	learn: 0.7662698	total: 46m 28s	remaining: 2m 36s
4734:	learn: 0.7662238	total: 46m 29s	remaining: 2m 36s
4735:	learn: 0.7661912	total: 46m 30s	remaining: 2m 35s
4736:	learn: 0.7661485	total: 46m 31s	remaining:

4867:	learn: 0.7620358	total: 48m 10s	remaining: 1m 18s
4868:	learn: 0.7620034	total: 48m 11s	remaining: 1m 17s
4869:	learn: 0.7619808	total: 48m 12s	remaining: 1m 17s
4870:	learn: 0.7619551	total: 48m 13s	remaining: 1m 16s
4871:	learn: 0.7619246	total: 48m 14s	remaining: 1m 16s
4872:	learn: 0.7618917	total: 48m 14s	remaining: 1m 15s
4873:	learn: 0.7618717	total: 48m 15s	remaining: 1m 14s
4874:	learn: 0.7618516	total: 48m 16s	remaining: 1m 14s
4875:	learn: 0.7618394	total: 48m 16s	remaining: 1m 13s
4876:	learn: 0.7618143	total: 48m 17s	remaining: 1m 13s
4877:	learn: 0.7617817	total: 48m 18s	remaining: 1m 12s
4878:	learn: 0.7617725	total: 48m 19s	remaining: 1m 11s
4879:	learn: 0.7617398	total: 48m 19s	remaining: 1m 11s
4880:	learn: 0.7617153	total: 48m 20s	remaining: 1m 10s
4881:	learn: 0.7617004	total: 48m 21s	remaining: 1m 10s
4882:	learn: 0.7616771	total: 48m 22s	remaining: 1m 9s
4883:	learn: 0.7616441	total: 48m 22s	remaining: 1m 8s
4884:	learn: 0.7616114	total: 48m 23s	remaining: 1

In [0]:
pred_y = model.predict(X_test)

In [0]:
# 최종 결과
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

ValueError: Found input variables with inconsistent numbers of samples: [5149, 200]

In [0]:
test

,rank_g,you_rank_g,previous_ranking,genre,season,sex,song_type,active_type,title_song,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view
43,10,10,10,All,spring,female,K_POP,Group,0,0.547337,0.240385,0.700,0.458548,0.0,0.403232,0.000000,0.0,0.0,0.0,0.0
50,3,10,2,Dance,spring,female,K_POP,Group,1,0.502959,0.019231,0.700,0.458548,0.0,0.551930,0.305731,0.0,0.0,0.0,0.0
149,10,10,10,Drama,spring,male,OST,Solo,1,0.547337,0.384615,0.270,0.395342,0.0,0.613522,0.503213,0.0,0.0,0.0,0.0
431,10,10,10,Electronica,spring,male,POP,Project,1,0.497041,0.471154,0.053,0.780646,0.0,0.569868,0.000000,0.0,0.0,0.0,0.0
496,2,10,2,Pop,spring,female,POP,Solo,1,0.476331,0.634615,0.000,0.664783,0.0,0.529333,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16733,9,10,10,Ballad,spring,male,K_POP,Solo,1,0.739645,0.000000,0.100,0.056849,0.0,0.660567,0.000000,0.0,0.0,0.0,0.0
16990,10,10,10,Rap/Hiphop,spring,female,K_POP,Solo,1,0.529586,0.682692,0.150,0.265167,0.0,0.477572,0.000000,0.0,0.0,0.0,0.0
17149,10,10,10,Ballad,spring,male,K_POP,Solo,0,0.529586,0.240385,0.030,0.230701,0.0,0.660567,0.170842,0.0,0.0,0.0,0.0
17243,6,10,6,Ballad,spring,male,K_POP,Solo,1,0.615385,0.423077,0.030,0.230701,0.0,0.660567,0.170842,0.0,0.0,0.0,0.0


In [0]:
pred_y = model.predict(test.drop(columns = ['rank_g']))

In [0]:
print("F1 : %.3f" % f1_score(pred_y, test['rank_g'], average = 'micro'))

F1 : 0.765


# Xgboost

In [0]:
from xgboost import XGBClassifier

In [0]:
# XGB는 생각보다 성능이 낮다... 아마 파라미터 조정을 해야될듯...
model = XGBClassifier(n_estimators = 500,learning_rate = 0.05)

In [0]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

In [0]:
score.mean()

0.6954003804672746

In [0]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [0]:
pred_y = model.predict(X_test)
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.687


In [0]:
import joblib

In [0]:
clf_from_joblib = joblib.load('final_model.pkl') 

In [0]:
pred_y = clf_from_joblib.predict(X_test)

In [0]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.719
